Next up: Add cases for circular, parabolic, and hyperbolic orbits. Check Danby, there is a new formula for r(v) etc.


Taking Chris's advice, I need to update this code so that it can handle longer inputs depending on the decimal value. Newton's method fails for some level of this, so either want to switch to bi section method, or just add some dumb and over the top condition that influences how many data points I do for Newton's where it corresponds to the desired e of the orbit times the number of sig figs in the anomaly to be converted. 

In [1]:
import numpy as np
import math as mth

In [2]:
def length(v):
    return mth.sqrt(np.dot(v, v))

def angle(v1, v2):

    return mth.acos(np.dot(v1, v2) / (length(v1) * length(v2)))

def normalize(x):
    xnorm = []
    mag = 0
    for i in range(len(x)):
        mag += x[i]**2    
    for j in range(len(x)):
        xnorm.append(x[j]/np.sqrt(mag))
        
    return xnorm

In [16]:
def OE2cart(a,e,i,O,w,v):
    #first to put it in terms of radians...
    i = np.deg2rad(i)    
    O = np.deg2rad(O)
    w = np.deg2rad(w)            
    v = np.deg2rad(v)
    
    
    GM=1
    if e<1:
        p=a*(1-e**2)
#start with work in a perifocal coordinate frame... 
        r=p/(1+e*np.cos(v))
    #magnitude of r right here...
    #now magnitude of v...
#v=sqrt(GM/p)[-sinvX+(e+cosv)Y]

        velperifocal=np.sqrt(GM*(2/r-1/a)) 
    
        rP=r*np.cos(v)
        rQ=r*np.sin(v)
        rperifocal=[[rP],[rQ],[0]]
        vP=np.sqrt(GM/p)*(-np.sin(v))
        vQ=np.sqrt(GM/p)*(e+np.cos(v))
        vperifocal = [[vP],[vQ],[0]]
    #obtained from Fundamental of Astrodynamics, using a matrix transformation from perifocal to Cartesion coord systems. 
        R11=(np.cos(O)*np.cos(w)-np.sin(O)*np.sin(w)*np.sin(i))
        R12 = -np.cos(O)*np.sin(w)-np.sin(O)*np.cos(w)*np.cos(i)
        R13 = np.sin(O)*np.sin(i)
        R21 = np.sin(O)*np.cos(w) + np.cos(O)*np.sin(w)*np.cos(i)
        R22 = -np.sin(O)*np.sin(w) + np.cos(O)*np.cos(w)*np.cos(i)
        R23 = -np.cos(O)*np.sin(i)
        R31 = np.sin(w)*np.sin(i)
        R32 = np.cos(w)*np.sin(i)
        R33 = np.cos(i)
        R=[[R11,R12,R13],[R21,R22,R23],[R31,R32,R33]]
     
        rcart=[[0],[0],[0]]
        vcart=[[0],[0],[0]]
        for i in range(len(R)):
             for j in range(len(rperifocal[0])):
                for k in range(len(rperifocal)):
                    rcart[i][j] += R[i][k] * rperifocal[k][j]
                    vcart[i][j] +=R[i][k] *vperifocal[k][j]
    else:
        print("testing the hyperbolic/parabolic section")
        p=a*(1-e**2)
        print("p = " + str(p))
      #  r=p/(-1+e*np.cos(v))
        r=p/(1+e*np.cos(v))
        
        print("r = " + str(r))
        velperifocal=np.sqrt(GM*(2/r-1/a)) 
    
        rP=r*np.cos(v)
        rQ=r*np.sin(v)
        rperifocal=[[rP],[rQ],[0]]
        vP=np.sqrt(GM/p)*(-np.sin(v))
        vQ=np.sqrt(GM/p)*(e+np.cos(v))
        vperifocal = [[vP],[vQ],[0]]
    #obtained from Fundamental of Astrodynamics, using a matrix transformation from perifocal to Cartesion coord systems. 
        R11=(np.cos(O)*np.cos(w)-np.sin(O)*np.sin(w)*np.sin(i))
        R12 = -np.cos(O)*np.sin(w)-np.sin(O)*np.cos(w)*np.cos(i)
        R13 = np.sin(O)*np.sin(i)
        R21 = np.sin(O)*np.cos(w) + np.cos(O)*np.sin(w)*np.cos(i)
        R22 = -np.sin(O)*np.sin(w) + np.cos(O)*np.cos(w)*np.cos(i)
        R23 = -np.cos(O)*np.sin(i)
        R31 = np.sin(w)*np.sin(i)
        R32 = np.cos(w)*np.sin(i)
        R33 = np.cos(i)
        R=[[R11,R12,R13],[R21,R22,R23],[R31,R32,R33]]
     
        rcart=[[0],[0],[0]]
        vcart=[[0],[0],[0]]
        for i in range(len(R)):
             for j in range(len(rperifocal[0])):
                for k in range(len(rperifocal)):
                    rcart[i][j] += R[i][k] * rperifocal[k][j]
                    vcart[i][j] +=R[i][k] *vperifocal[k][j]
    print("x,y,z,vx,vy,vz")
    return rcart,vcart

In [20]:
OE2cart(1,1,0,0,0,0)

testing the hyperbolic/parabolic section
p = 0
r = 0.0


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:53: RuntimeWarning: divide by zero encountered in double_scalars


ZeroDivisionError: division by zero

In [21]:
def cart2OE(x,y,z,vx,vy,vz):
    #first the things that will be used later... 
    GM=1
    r=np.array([x,y,z])
    rmag = length(r)
    v=np.array([vx,vy,vz])
    vmag = length(v)
    h=np.cross(r,v)
    I = np.array([1,0,0])  #x axis vector 
    K = np.array([0,0,1])  #z axis of this setup
    n = np.cross(K,h) #node vector, should be zero when uninclined
    nmag = length(n)  #magnitude of node vector, use this to decide if equatorial orbit or not. 
    #a
    a = 1/((2/rmag)-(vmag**2/GM))
    #e
    evec=np.cross(v,h)/GM-normalize(r) 
    e=length(evec)
    #i
    hmag = length(h)
    hnorm = normalize(h)
    i=np.arccos(hnorm[2])  #normalized w z axis, distance btw... seems to pass every test so far!
    #O
    if i>0 and i<np.pi:
        if n[1]>=0:
            O = np.arccos(np.dot(n,I)/length(n))
        elif n[1]<0:
            O = 2*np.pi - np.arccos(np.dot(n,I)/length(n))
    else:
        O = 0
    #w
    if i > 0 and i < np.pi: 
        if evec[2]>=0:
            w = np.arccos(np.dot(n,evec)/(length(evec)*length(n)))
        elif evec[2]<0:
            w = 2*np.pi - np.arccos(np.dot(n,evec)/(length(evec)*length(n)))
    else:
        if h[2]>=0:
            w = 2*np.pi + np.arctan2(evec[1],evec[0])
            if w >= 2*np.pi:
                w = w - 2*np.pi
        else: 
            w = 2*np.pi - np.arctan2(evec[1],evec[0])
            if w >=2*np.pi:
                w = w - 2*np.pi
    #v
    if np.dot(r,v)>0:
        v = np.arccos(np.dot(evec,r)/(length(evec)*length(r)))
    elif np.dot(r,v)<0:
        v = 2*np.pi - np.arccos(np.dot(I,r)/(length(I)*length(r)))
    
  #  print("h = " + str(h))
    return a,e,np.rad2deg(i),np.rad2deg(O),np.rad2deg(w),np.rad2deg(v)

In [22]:
aa,ee,ii,OO,ww,vv = cart2OE(-.1,8,0,.471,-4.23,0)

In [23]:
OE2cart(aa,ee,ii,OO,ww,vv)

testing the hyperbolic/parabolic section
p = 11.189024999999996
r = 13.597980999
x,y,z,vx,vy,vz


([[-0.72197837570406254], [13.578800921793361], [-1.662923508513337e-15]],
 [[0.4706016761297841], [-4.2178638254351792], [5.165393433058602e-16]])